In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import sqlite3

PATH = 'chromedriver'

In [2]:
driver = webdriver.Chrome(PATH)

# Function Solution

In [3]:
#return the df
def get_data(url, findId, headerIndex):
    driver.get(url)
    res = driver.find_element(By.ID, findId)
    rows = res.find_elements(By.TAG_NAME, "tr")
    #get column names from first row
    cols = []
    for ele in rows[headerIndex].find_elements(By.TAG_NAME, "th"):
        cols.append(ele.text)
    cols[1:]
    #create dataframe and append table rows to it
    stats = pd.DataFrame(columns = cols[1:])
    for row in rows:
        player = []
        for td in row.find_elements(By.TAG_NAME, "td"):
            player.append(td.text)
        #try except is needed here so that header rows don't throw an error when added to the df and are instead skipped
        try:
            stats.loc[len(stats)] = player
        except:
            continue
    return(stats)

In [46]:
gsc_stats  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_gca', 1)
defense_stats = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_defense', 1)
keeper_stats = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_keeper', 1)
misc_stats = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_misc', 1)
pass_stats = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_passing', 1)
possession_stats = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_possession', 1)
shooting_stats = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_shooting', 1)
time_stats = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_playing_time', 1)

In [4]:
east_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results114991Eastern-Conference_overall', 0)
west_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results114991Western-Conference_overall', 0)

squad_goalkeeping = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_squads_keeper_for', 1)
squad_gsc  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_squads_gca_for', 1)
squad_defense = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_squads_defense_for', 1)
squad_misc = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_squads_misc_for', 1)
squad_pass = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_squads_passing_for', 1)
squad_possession = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_squads_possession_for', 1)
squad_shooting = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_squads_shooting_for', 1)
squad_time = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_squads_playing_time_for', 1)


In [5]:
#create function to add squad name to each squad df
squad_dict = {0:'Atlanta Utd', 1: 'Austin FC', 2:'CF Montréal', 3:'Charlotte FC', 4: 'Chicago Fire', 5:'Colorado Rapids', 6:'Columbus Crew', 7:'D.C. United', 8:'FC Cincinnati', 9:'FC Dallas', 10:'Houston Dynamo', 11:'Inter Miami', 12:'LA Galaxy', 13:'Los Angeles FC', 14:'Minnesota Utd', 15:'Nashville', 16:'New England', 17:'NY Red Bulls', 18:'NYCFC', 19:'Orlando City', 20:'Philadelphia', 21:'Portland Timbers', 22:'Real Salt Lake', 23:'San Jose', 24:'Seattle', 25:'Sporting KC', 26:'Toronto FC', 27:'Vancouver'}

def add_squads(df):
    df.reset_index(inplace = True)
    df['squad'] = df['index'].apply(lambda x: squad_dict[x])
    df.drop('index', inplace = True, axis = 1)

In [6]:
add_squads(squad_goalkeeping)
add_squads(squad_gsc)
add_squads(squad_defense)
add_squads(squad_misc)
add_squads(squad_pass)
add_squads(squad_possession)
add_squads(squad_shooting)
add_squads(squad_time)

In [7]:
squads = pd.concat([east_squads, west_squads]).reset_index(drop=True)

In [51]:
squads

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,Philadelphia,23,12,9,2,40,15,+25,45,1.96,37.7,25.4,+12.3,+0.53,W W W W W,"17,912",Dániel Gazdag - 11,Andre Blake,
1,NYCFC,22,12,6,4,41,21,+20,42,1.91,44.9,26.2,+18.6,+0.85,W W W W D,"18,169",Valentín Castellanos - 13,Sean Johnson,
2,CF Montréal,22,11,3,8,36,36,0,36,1.64,31.2,23.1,+8.1,+0.37,L L W W D,"14,963",Romell Quioto - 9,Sebastian Breza,
3,NY Red Bulls,23,10,6,7,38,30,+8,36,1.57,34.9,31.0,+3.9,+0.17,W D L W L,"16,063",Lewis Morgan - 10,Carlos Coronel,
4,Orlando City,23,8,6,9,27,33,-6,30,1.30,26.7,32.8,-6.2,-0.27,W D D L L,"15,487",Ercan Kara - 7,Pedro Gallese,
5,Columbus Crew,21,7,9,5,27,22,+5,30,1.43,28.0,25.1,+2.9,+0.14,D W D W D,"19,455","Lucas Zelarayán, Derrick Etienne - 5",Eloy Room,
6,Charlotte FC,23,9,2,12,28,31,-3,29,1.26,25.2,28.3,-3.2,-0.14,W W L L W,"35,916",Karol Świderski - 6,Kristijan Kahlina,
7,Inter Miami,23,8,5,10,27,37,-10,29,1.26,27.8,31.6,-3.8,-0.17,L W L D W,"12,305",Leonardo Campana - 8,Drake Callender,
8,FC Cincinnati,23,7,8,8,37,42,-5,29,1.26,36.9,35.5,+1.3,+0.06,D D L D D,"21,793",Brandon Vazquez - 13,Roman Celentano,
9,Chicago Fire,23,7,6,10,24,28,-4,27,1.17,29.1,30.3,-1.2,-0.05,L W W W D,"15,356",Xherdan Shaqiri - 4,Gabriel Slonina,


# Player Database prep and creation

In [52]:
defense_stats.reset_index()

player_ids = dict(zip(time_stats.Player, time_stats.index))

In [53]:
player_ids
#add ids to each player df

{'Paxten Aaronson': 0,
 'Lalas Abubakar': 1,
 'Nicolás Acevedo': 2,
 'Ifunanyachi Achara': 3,
 'Bryan Acosta': 4,
 'George Acosta': 5,
 'Kellyn Acosta': 6,
 'Luciano Acosta': 7,
 'Mo Adams': 8,
 'Samuel Adeniran': 9,
 'Harrison Afful': 10,
 'Samuel Afriyie Owusu': 11,
 'William Agada': 12,
 'Oskar Ågren': 13,
 'Daniel Aguirre': 14,
 'Ali Ahmed': 15,
 'Arnaud Ake': 16,
 'Tesho Akindele': 17,
 'Ayo Akinola': 18,
 'Jordy Alcivar': 19,
 'Caio Alexandre': 20,
 'Tony Alfaro': 21,
 'Noah Allen': 22,
 'Thiago Almada': 23,
 'Osvaldo Alonso': 24,
 'Jozy Altidore': 25,
 'Efrain Alvarez': 26,
 'Luis Amarilla': 27,
 'Frankie Amaya': 28,
 'Mikey Ambrose': 29,
 'Malte Amundsen': 30,
 'Max Anchor': 31,
 'Sebastian Anderson': 32,
 'José Andrés Martínez': 33,
 'Jalil Anibaba': 34,
 'Themi Antonoglou': 35,
 'Brian Anunga': 36,
 'Cristian Arango': 37,
 'César Araujo': 38,
 'Julian Araujo': 39,
 'Luiz Araújo': 40,
 'Daniel Armando Ríos': 41,
 'Adam Armour': 42,
 'Xavier Arreaga': 43,
 'Kervin Arriaga': 44,

In [54]:
def set_id(df):
    df['player_id'] = df['Player'].apply(lambda x: player_ids[x])
    df.drop('Player', axis=1, inplace = True)



In [55]:
set_id(gsc_stats)
set_id(defense_stats)
set_id(keeper_stats)
set_id(pass_stats)
set_id(possession_stats)
set_id(shooting_stats)
set_id(misc_stats)
time_stats['player_id'] = time_stats['Player'].apply(lambda x: player_ids[x])
players = time_stats[['player_id', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s']]
time_stats.drop('Player', axis=1, inplace = True)

In [56]:
#Remove calculated and redundant fields from dataframes
gsc_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', 'SCA', 'SCA90', 'GCA', 'GCA90'], axis =1, inplace = True)
defense_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Tkl%', '%', 'Matches', 'Tkl+Int'], axis=1, inplace = True)
keeper_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Min', '90s', 'GA90', 'Save%', 'CS%', 'Matches', 'W', 'D', 'L'], axis=1, inplace = True)
pass_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Cmp%', 'Cmp%', 'Cmp%', 'Cmp%', 'A-xA', '1/3', 'Matches'], axis=1, inplace = True)
possession_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Touches', 'Succ%', '1/3', 'Rec%', 'Matches'], axis=1, inplace = True)
shooting_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches'], axis=1, inplace = True)
misc_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Int', 'TklW', 'Won%', 'Matches'], axis=1, inplace = True)
time_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', 'Mn/MP', 'Min%', '90s', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'Matches'], axis=1, inplace = True)

In [73]:
# Rename columns for database upload, drop calculated columns that weren't dropped earlier due to column name overlap
keeper_stats.rename(columns = {'Starts':'starts', 'GA':'ga', 'SoTA':'sota', 'Saves':'saves', 'CS':'cs', 'PKatt':'pkAtt', 'PKA':'pka', 'PKsv':'pksv', 'PKm':'pkm', 'player_id':'player_id'}, inplace = True)
gsc_stats.set_axis(['scPassLive', 'scPassDead', 'scDrib', 'scSh', 'scFld', 'scDef', 'gcPassLive', 'gcPassDead', 'gcDrib', 'gcSh', 'gcFld', 'gcDef', 'player_id' ], axis=1, inplace=True)
defense_stats.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'tklAttVsDrib', 'past', 'press', 'pressW', 'pressDefThird', 'pressMidThird', 'pressAttThird', 'blk', 'blkSh', 'blkSot', 'blkPass', 'int', 'clr', 'err', 'player_id'], axis=1, inplace = True)
pass_stats.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'player_id' ], axis=1, inplace=True)
pass_stats.drop(['totCmp', 'totAtt'], axis = 1, inplace = True)
possession_stats.set_axis(['defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'liveTouches', 'succDrib', 'attDrib', 'playersBeat', 'megs', 'carries', 'totDist', 'prgDist', 'progCarries', 'cpa', 'mis', 'tackled', 'passTarg', 'passRec', 'progPassRec', 'player_id'], axis=1, inplace=True)
shooting_stats.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pk', 'pkAtt', 'xG', 'npxG', 'player_id'], axis=1, inplace=True)
misc_stats.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'PKcon', 'og', 'recov', 'airW', 'airL', 'player_id'], axis=1, inplace=True)
time_stats.set_axis(['mp', 'min', 'starts', 'mn/start', 'compl', 'subs', 'mn/sub', 'unSub', 'ppm', 'onG', 'onGa', 'on-off', 'onxG', 'onxGa', 'xon-off', 'player_id'], axis=1, inplace=True)
players.set_axis(['id', 'name', 'nation', 'pos', 'squad', 'age', 'born', '90s'], axis = 1, inplace = True)

ValueError: Length mismatch: Expected axis has 15 elements, new values have 17 elements

In [76]:
#add to db next
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    players.to_sql('players', conn, if_exists='replace', index = False)
    keeper_stats.to_sql('playerKeepers', conn, if_exists='replace', index = False)
    defense_stats.to_sql('playerDefense', conn, if_exists='replace', index = False) 
    gsc_stats.to_sql('playerGsc', conn, if_exists='replace', index = False)
    misc_stats.to_sql('playerMisc', conn, if_exists='replace', index = False)  
    pass_stats.to_sql('playerPassing', conn, if_exists='replace', index = False)
    possession_stats.to_sql('playerPossession', conn, if_exists='replace', index = False)
    shooting_stats.to_sql('playerShooting', conn, if_exists='replace', index = False)    
    time_stats.to_sql('playerTime', conn, if_exists='replace', index = False) 

# Squad Database Prep and Creation

In [8]:
#we don't need to set ID here as squad name is an adaquate primary key since it will never be duplicated
squads.drop(['Pts/MP', 'Pts', 'GD', 'GF', 'xGD/90', 'xGD', 'Top Team Scorer', 'Notes', 'xG'], axis=1, inplace = True)
squad_goalkeeping.drop(['# Pl', 'MP', 'Starts', 'Min', '90s', 'GA', 'GA90', 'Save%', 'W', 'D', 'L', 'CS%'], axis = 1, inplace = True)
squad_gsc.drop(['# Pl', '90s', 'SCA90', 'GCA90', 'SCA', 'GCA'], axis = 1, inplace = True)
#drop first tkl after renaming
squad_defense.drop(['# Pl', '90s', 'Tkl%', '%', 'Tkl+Int'], axis = 1, inplace = True)
squad_misc.drop(['# Pl', '90s', 'Int', 'TklW', 'Won%'], axis = 1, inplace = True)
#drop first cmp after renaming
squad_pass.drop(['# Pl', '90s', 'Cmp%', 'A-xA', '1/3'], axis = 1, inplace = True)
#drop first # pl after renaming
squad_possession.drop([ '90s', 'Touches', 'Succ%', '1/3', 'Rec%'], axis = 1, inplace = True)
squad_shooting.drop(['# Pl', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], axis = 1, inplace = True)
squad_time.drop(['# Pl', 'MP', 'Mn/MP', 'Min%', '90s', 'Starts', 'Mn/Start', 'Compl', 'PPM', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'onG', 'onGA', 'onxG', 'onxGA'], axis = 1, inplace = True)

In [9]:
squads.set_axis(['squad', 'mp', 'w', 'd', 'l', 'ga', 'xGA', 'last5', 'attendence', 'goalkeeper'], axis=1, inplace=True)
squad_goalkeeping.set_axis(['sota', 'saves', 'cs', 'pkAtt', 'pkA', 'pkSv', 'pkM', 'squad'], axis=1, inplace=True)
squad_defense.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'attVsDrib', 'past', 'press', 'pressW', 'pressDefThird', 'pressMidThird', 'pressAttThird', 'blk', 'blkSh', 'blkSot', 'blkPass', 'int', 'clr', 'err', 'squad'], axis=1, inplace=True)
squad_misc.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'pkCon', 'og', 'recov', 'airW', 'airL', 'squad'], axis=1, inplace=True)
squad_pass.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'squad'], axis=1, inplace=True)
squad_possession.set_axis(['# Pl', 'poss', 'defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'live', 'dribW', 'dribAtt', 'playersBeat', 'megs', 'carries', 'totDist', 'prgDist', 'progCarries', 'cpa', 'mis', 'dis', 'targ', 'rec', 'progRec', 'squad' ], axis=1, inplace=True)
squad_possession.drop('# Pl', axis = 1, inplace = True)
squad_shooting.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pkW', 'pkAtt', 'xG', 'npxG', 'squad'], axis=1, inplace=True)
squad_time.set_axis(['age','min','subs','mn/sub','unsub', 'squad'], axis=1, inplace=True)
squad_gsc.set_axis(['scPassLive', 'scPassDead', 'scDrib', 'scSh', 'scFld', 'scDef', 'gcPassLive', 'gcPassDead', 'gcDrib', 'gcSh', 'gcFld', 'gcDef', 'squad' ], axis=1, inplace=True)


ValueError: Length mismatch: Expected axis has 23 elements, new values have 22 elements

In [16]:
#add to db next
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    squads.to_sql('squads', conn, if_exists='replace', index = False)
    squad_goalkeeping.to_sql('squadKeepers', conn, if_exists='replace', index = False)
    squad_defense.to_sql('squadDefense', conn, if_exists='replace', index = False) 
    squad_gsc.to_sql('squadGsc', conn, if_exists='replace', index = False)
    squad_misc.to_sql('squadMisc', conn, if_exists='replace', index = False)  
    squad_pass.to_sql('ayerPassing', conn, if_exists='replace', index = False)
    possession_stats.to_sql('playerPossession', conn, if_exists='replace', index = False)
    shooting_stats.to_sql('playerShooting', conn, if_exists='replace', index = False)    
    time_stats.to_sql('playerTime', conn, if_exists='replace', index = False) 

Index(['Age', 'Min', 'Subs', 'Mn/Sub', 'unSub', 'onG', 'onGA', 'onxG', 'onxGA',
       'squad'],
      dtype='object')

In [18]:
squads.set_axis(['squad', 'mp', 'w', 'd', 'l', 'ga', 'xGA', 'last5', 'attendence', 'goalkeeper'], axis=1, inplace=True)
squad_goalkeeping.set_axis(['sota', 'saves', 'cs', 'pkAtt', 'pkA', 'pkSv', 'pkM', 'squad'], axis=1, inplace=True)
squad_defense.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'attVsDrib', 'past', 'press', 'pressW', 'pressDefThird', 'pressMidThird', 'pressAttThird', 'blk', 'blkSh', 'blkSot', 'blkPass', 'int', 'clr', 'err', 'squad'], axis=1, inplace=True)
squad_misc.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'pkCon', 'og', 'recov', 'airW', 'airL', 'squad'], axis=1, inplace=True)
squad_pass.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'squad'], axis=1, inplace=True)
squad_possession.set_axis(['# Pl', 'poss', 'defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'live', 'dribW', 'dribAtt', 'playersBeat', 'megs', 'carries', 'totDist', 'prgDist', 'progCarries', 'cpa', 'mis', 'dis', 'targ', 'rec', 'progRec', 'squad' ], axis=1, inplace=True)
squad_possession.drop('# Pl', axis = 1, inplace = True)
squad_shooting.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pkW', 'pkAtt', 'xG', 'npxG', 'squad'], axis=1, inplace=True)
squad_time.set_axis(['age','min','subs','mn/sub','unsub', 'squad'], axis=1, inplace=True)


In [20]:
squad_gsc.columns

Index(['SCA', 'PassLive', 'PassDead', 'Drib', 'Sh', 'Fld', 'Def', 'GCA',
       'PassLive', 'PassDead', 'Drib', 'Sh', 'Fld', 'Def', 'squad'],
      dtype='object')

In [21]:
squad_gsc.drop(['SCA', 'GCA'], axis = 1, inplace = True)